In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x744a54485690>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x744a54486260>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Rashmi and I am a Data scientist")])

AIMessage(content="Hi Rashmi, it's nice to meet you!\n\nAs a large language model, I'm always interested in learning more about people who work in data science. What kind of projects are you working on these days?  Anything exciting? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 56, 'prompt_tokens': 22, 'total_tokens': 78, 'completion_time': 0.101818182, 'prompt_time': 0.00013712, 'queue_time': 0.019592559, 'total_time': 0.101955302}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2c956fa3-565c-4175-9680-70cb6d60b0c4-0', usage_metadata={'input_tokens': 22, 'output_tokens': 56, 'total_tokens': 78})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Rashmi and I am a Data Scientist"),
        AIMessage(content="Hi Rashmi, it's nice to meet you!\n\nAs a large language model, I'm always interested in learning more about people who work in data science. What kind of projects are you working on these days?  Anything exciting? 😊  \n\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You are Rashmi, and you are a Data Scientist!  \n\nIs there anything else you'd like to know or talk about?  Perhaps you'd like to discuss a data science project you're working on?  I'm here to chat! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 97, 'total_tokens': 156, 'completion_time': 0.107272727, 'prompt_time': 0.00349234, 'queue_time': 0.024435439, 'total_time': 0.110765067}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2a2d4fc6-c9b5-4132-a91e-f795484f4b7d-0', usage_metadata={'input_tokens': 97, 'output_tokens': 59, 'total_tokens': 156})

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [10]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Rashmi and I am a Data Scientist")],
    config=config
)

In [12]:
response.content

"Hi Rashmi, it's nice to meet you!  \n\nBeing a Data Scientist is a fascinating field. What kind of work do you enjoy doing the most?  Do you have a particular area of specialization? \n\nI'm always eager to learn more about what people in different professions do.\n"

In [13]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Rashmi.  😊 \n\nI remember that from our first interaction!  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 101, 'total_tokens': 133, 'completion_time': 0.058181818, 'prompt_time': 0.00325448, 'queue_time': 0.022769308999999998, 'total_time': 0.061436298}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0dc6d4ca-2cad-4582-9389-b0eaf149896a-0', usage_metadata={'input_tokens': 101, 'output_tokens': 32, 'total_tokens': 133})

In [14]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'd be happy to know! 😊\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is sona")],
    config=config1
)
response.content

"Hello Sona, it's nice to meet you!  \n\nIs there anything I can help you with today? 😊 \n"

In [16]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Sona!  I remember now. 😊  How can I help you?  \n'

In [17]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [18]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Rash")]})

AIMessage(content="Hi Rash! It's nice to meet you. \n\nI'm happy to help with any questions you have. Just ask away! 😊  What can I do for you today?  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 31, 'total_tokens': 74, 'completion_time': 0.078181818, 'prompt_time': 0.000415598, 'queue_time': 0.024211611, 'total_time': 0.078597416}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-795a426d-19c5-48e3-ba21-d8baf38b154c-0', usage_metadata={'input_tokens': 31, 'output_tokens': 43, 'total_tokens': 74})

In [19]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [20]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Rash")],
    config=config
)

response

AIMessage(content="Hi Rash, it's nice to meet you! 👋 \n\nI'm ready to answer your questions to the best of my ability.  Just ask away! 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 31, 'total_tokens': 70, 'completion_time': 0.070909091, 'prompt_time': 0.000399588, 'queue_time': 0.023425971, 'total_time': 0.071308679}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-03696ead-0895-4a05-9904-0338050414f1-0', usage_metadata={'input_tokens': 31, 'output_tokens': 39, 'total_tokens': 70})

In [21]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Rash.  I remember! 😊  \n\nDo you have any other questions for me? \n'

In [22]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [23]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Rash")],"language":"Kannada"})
response.content

'ನಮಸ್ತೆ ರಾಶ್!  \n\nನಾನು ನಿಮಗೆ ಸಹಕರಿಸಲು ಸಿದ್ಧಿದ್ದೇನೆ.  \nನಿಮಗಾಗಿ ಏನಾದರೂ ಮಾಡಬೇಕೆಂದರೆ ಕೇಳಿ.\n'

In [28]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [30]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Rash")],"language":"Kannada"},
    config=config
)
repsonse.content

'ನಮಸ್ತೆ Rash!  😄  ನನ್ನೊಂದಿಗೆ ಏನನ್ನೋ ಮಾತನಾಡಬೇಕು, ಅಥವಾ ಯಾವುದಾದರೂ ಕೆಲಸವನ್ನು ಮಾಡಬೇಕು?  ನಾನು ನಿಮ್ಮ ಸಹಾಯ ಮಾಡಲು ಸಿದ್ಧವಾಗಿದೆ. \n'

In [31]:
repsonse.content

'ನಮಸ್ತೆ Rash!  😄  ನನ್ನೊಂದಿಗೆ ಏನನ್ನೋ ಮಾತನಾಡಬೇಕು, ಅಥವಾ ಯಾವುದಾದರೂ ಕೆಲಸವನ್ನು ಮಾಡಬೇಕು?  ನಾನು ನಿಮ್ಮ ಸಹಾಯ ಮಾಡಲು ಸಿದ್ಧವಾಗಿದೆ. \n'

In [26]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Kannada"},
    config=config,
)

In [27]:
response.content

'ನಿಮ್ಮ ಹೆಸರು Rash. 😊 \n\n\n'

In [1]:
from langchain_openai import ChatOpenAI

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)

In [9]:
from langchain_core.messages import SystemMessage,trim_messages,AIMessage,HumanMessage
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter="gpt-4-0125",
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage("you're a good assistant, you always respond with a joke."),
    HumanMessage("i wonder why it's called langchain"),
    AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
    ),
    HumanMessage("and who is harrison chasing anyways"),
    AIMessage(
        "Hmmm let me think.\n\nWhy, he's probably chasing after the last cup of coffee in the office!"
    ),
    HumanMessage("what do you call a speechless parrot"),
]


trim_messages(
    messages,
    # Keep the last <= n_count tokens of the messages.
    strategy="last",
    # Remember to adjust based on your model
    # or else pass a custom token_encoder
    token_counter=ChatOpenAI(model="gpt-4o"),
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    # Remember to adjust based on the desired conversation
    # length
    max_tokens=45,
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    start_on="human",
    # Most chat models expect that chat history ends with either:
    # (1) a HumanMessage or
    # (2) a ToolMessage
    end_on=("human", "tool"),
    # Usually, we want to keep the SystemMessage
    # if it's present in the original history.
    # The SystemMessage has special instructions for the model.
    include_system=True,
    allow_partial=False,
)

[SystemMessage(content="you're a good assistant, you always respond with a joke.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what do you call a speechless parrot', additional_kwargs={}, response_metadata={})]

In [10]:
llm = ChatOpenAI(model="gpt-4o")

# Notice we don't pass in messages. This creates
# a RunnableLambda that takes messages as input
trimmer = trim_messages(
    token_counter=llm,
    # Keep the last <= n_count tokens of the messages.
    strategy="last",
    # When token_counter=len, each message
    # will be counted as a single token.
    # Remember to adjust for your use case
    max_tokens=45,
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    start_on="human",
    # Most chat models expect that chat history ends with either:
    # (1) a HumanMessage or
    # (2) a ToolMessage
    end_on=("human", "tool"),
    # Usually, we want to keep the SystemMessage
    # if it's present in the original history.
    # The SystemMessage has special instructions for the model.
    include_system=True,
)

chain = trimmer | llm
chain.invoke(messages)



AIMessage(content='A *polly*-got-nothing-to-say!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 32, 'total_tokens': 45, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 'stop', 'logprobs': None}, id='run-a4d65fe7-e73f-4050-a6b0-b9ce5127b0ee-0', usage_metadata={'input_tokens': 32, 'output_tokens': 13, 'total_tokens': 45, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant, you always respond with a joke.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what do you call a speechless parrot', additional_kwargs={}, response_metadata={})]

In [12]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chat_history = InMemoryChatMessageHistory(messages=messages[:-1])


def dummy_get_session_history(session_id):
    if session_id != "1":
        return InMemoryChatMessageHistory()
    return chat_history


llm = ChatOpenAI(model="gpt-4o")

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=llm,
    # Usually, we want to keep the SystemMessage
    # if it's present in the original history.
    # The SystemMessage has special instructions for the model.
    include_system=True,
    # Most chat models expect that chat history starts with either:
    # (1) a HumanMessage or
    # (2) a SystemMessage followed by a HumanMessage
    # start_on="human" makes sure we produce a valid chat history
    start_on="human",
)

chain = trimmer | llm
chain_with_history = RunnableWithMessageHistory(chain, dummy_get_session_history)
chain_with_history.invoke(
    [HumanMessage("what do you call a speechless parrot")],
    config={"configurable": {"session_id": "1"}},
)

AIMessage(content='A "polygon"!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 32, 'total_tokens': 37, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'stop', 'logprobs': None}, id='run-6773e118-70be-42d0-87de-614262ae2346-0', usage_metadata={'input_tokens': 32, 'output_tokens': 5, 'total_tokens': 37, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})